## install modules

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import os

In [2]:
%%script echo skipping

os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")

skipping


## code

In [3]:
info = foz.load_zoo_dataset_info('open-images-v7')
info.downloaded_splits

{'validation': <fiftyone.zoo.datasets.ZooDatasetSplitInfo at 0x7f3dd2e8a9e0>,
 'train': <fiftyone.zoo.datasets.ZooDatasetSplitInfo at 0x7f3dd2e8afb0>}

In [4]:
classes_used = ["Cat"]

# validation_dataset = foz.load_zoo_dataset(
#     "coco-2017",
#     split="validation",
#     label_types=["detections"],
#     classes=classes_used
# )

# validation_dataset_view = validation_dataset.filter_labels(
#     "ground_truth",
#     fo.ViewField("label").is_in(classes_used),
# )

# train_dataset = foz.load_zoo_dataset(
#     "coco-2017",
#     split="train",
#     label_types=["detections"],
#     classes=classes_used
# )

# train_dataset_view = train_dataset.filter_labels(
#     "ground_truth",
#     fo.ViewField("label").is_in(classes_used),
# )

train_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=classes_used,
    max_samples=484
)

train_dataset_view = train_dataset.filter_labels(
    "ground_truth",
    fo.ViewField("label").is_in(classes_used),
)

validation_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections"],
    classes=classes_used,
    max_samples=124
)

validation_dataset_view = validation_dataset.filter_labels(
    "ground_truth",
    fo.ViewField("label").is_in(classes_used),
)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading 'open-images-v7' split 'train'
 100% |█████████████████| 484/484 [564.8ms elapsed, 0s remaining, 856.9 samples/s]      
Dataset 'open-images-v7-train-484' created
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading 'open-images-v7' split 'validation'
 100% |█████████████████| 124/124 [177.1ms elapsed, 0s remaining, 704.8 samples/s] 
Dataset 'open-images-v7-validation-124' created


In [5]:
for sample in validation_dataset:
    print(sample['ground_truth'])
    break

<Detections: {
    'detections': [
        <Detection: {
            'id': '65a76a554777eadf0c10a2e2',
            'attributes': {},
            'tags': [],
            'label': 'Cat',
            'bounding_box': [0.0, 0.0, 1.0, 1.0],
            'mask': None,
            'confidence': None,
            'index': None,
            'IsOccluded': False,
            'IsTruncated': True,
            'IsGroupOf': False,
            'IsDepiction': False,
            'IsInside': False,
        }>,
    ],
}>


In [6]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        # transforms.append(T.RandomRotation(30))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    # transforms.append(T.Resize((256,256)))
    return T.Compose(transforms)

In [7]:
import torch
from PIL import Image
from torchvision import tv_tensors
# import fiftyone.utils.coco as fouc
from torchvision.io import read_image, ImageReadMode

class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.

    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for
            training or testing
        transforms (None): a list of PyTorch transforms to apply to images
            and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset
            that contains the desired labels to load
        classes (None): a list of class strings that are used to define the
            mapping between class names and indices. If None, it will use
            all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        # self.filtered_indices = []
        # for sample in fiftyone_dataset:
        #     detections = sample[gt_field].detections
        #     if detections and any(self.is_valid_box(det.bounding_box) for det in detections):
        #         self.filtered_indices.append(sample.filepath)

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    # @staticmethod
    # def is_valid_box(box):
    #     # Check if the bounding box is valid (non-zero area)
    #     x1, y1, width, height = box
    #     x_min, x_max = min(x1,x2), max(x1,x2)
    #     y_min, y_max = min(y1,y2), max(y1,y2)
    #     return (x_max > x_min) and (y_max > y_min)
    
    def __getitem__(self, idx):
        # img_path = self.img_paths[idx]
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        #img = Image.open(img_path).convert("RGB")
        img = read_image(img_path, mode=ImageReadMode.RGB)
        img_height, img_width = img.size()[1:]        

        boxes = []
        labels = []
        areas = []
        isoccluded = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            box = list(det.bounding_box)
            #corrected_box = box.clone()

            # fiftyone dataset stores bbox as [0,1] values, and as XYWH format?
            x,y,width,height = box
            absolute_box = [x*img_width, y*img_height, (x+width)*img_width, (y+height)*img_height]
            
            # box[0], box[2] = min(box[0], box[2]), max(box[0], box[2])
            # box[1], box[3] = min(box[1], box[3]), max(box[1], box[3])
            # absolute_box = [box[0]*img_width, box[1]*img_height, box[2]*img_width, box[3]*img_height]

            area = (absolute_box[2]-absolute_box[0])*(absolute_box[3]-absolute_box[1])
            # area = (box[2]-box[0])*(box[3]-box[1])
            
            # if box[2] > box[0] and box[3] > box[1]: # make sure the box is valid
            boxes.append(absolute_box)
            labels.append(category_id)
            # isoccluded.append(det.IsOccluded)
            areas.append(area)
            iscrowd.append(0)

        target = {}
        # target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=(img_height, img_width), dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        # target["attributes"] = torch.as_tensor(isoccluded, dtype=torch.int64)
        target["image_id"] = idx #torch.as_tensor([idx])
        target["area"] = torch.as_tensor(areas, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        img = tv_tensors.Image(img)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        # return len(self.filtered_indices)
        return len(self.img_paths)

    def get_classes(self):
        return self.classes


        # image_paths = dataset.values("filepath")
        # sample = dataset[image_paths[1]]
        # print(sample)
        # detections = sample["ground_truth"].detections
        # metadata = sample.metadata
        # for det in detections:
        #     # category_id = self.labels_map_rev[det.label]
        #     category_id = 0
        #     print(det.label)
        #     print(det.IsOccluded)
        #     print(det.bounding_box)
        #     print(type(det.bounding_box))
            # coco_obj = fouc.COCOObject.from_label(
            #     det, metadata, category_id=category_id,
            # )
            #areas.append(det.area)

In [8]:
# import os

filepaths = train_dataset_view.values('filepath')
print(filepaths[0])
sample = train_dataset_view[filepaths[0]]

img = read_image(filepaths[0], mode=ImageReadMode.RGB)
img_height, img_width = img.size()[1:]        

boxes = []

#print(sample)
dets = sample['ground_truth'].detections
for det in dets:
    # print(det['bounding_box'])
    box = list(det.bounding_box)
    absolute_box = [box[0]*img_width, box[1]*img_height, box[2]*img_width, box[3]*img_height]
    print(box)
    print(absolute_box)
    boxes.append(absolute_box)

tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=(img_height, img_width), dtype=torch.float32)

# # Set the CUDA_LAUNCH_BLOCKING environment variable
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/home/zhiyi/fiftyone/open-images-v7/train/data/0000de486dc6c49f.jpg
[0.019608, 0.02451, 0.978758, 0.952614]
[20.078592, 25.09824, 1002.248192, 975.476736]


BoundingBoxes([[  20.0786,   25.0982, 1002.2482,  975.4767]], format=BoundingBoxFormat.XYXY, canvas_size=(1024, 1024))

In [9]:
from torch.utils.data import DataLoader
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import math
import sys

import utils # downloaded from github earlier

from torch.utils.data import random_split

# image_transforms = transforms.Compose([transforms.ToTensor()])
# image_transforms = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()])
# train_dataset = FiftyOneTorchDataset(train_dataset_view, transforms=image_transforms, classes=classes_used)
# validation_dataset = FiftyOneTorchDataset(validation_dataset_view, transforms=image_transforms, classes=classes_used)

train_dataset = FiftyOneTorchDataset(train_dataset_view, transforms=get_transform(train=True), classes=classes_used)
validation_dataset = FiftyOneTorchDataset(validation_dataset_view, transforms=get_transform(train=False), classes=classes_used)

# Define the ratio for splitting (e.g., 80% train, 20% validation)
# train_ratio = 0.8
# total_samples = len(torch_dataset)
# train_size = int(train_ratio * total_samples)
# print("train_size:", train_size)
# validation_size = total_samples - train_size
# print("validation_size:", validation_size)

# # Use random_split to create training and validation datasets
# train_dataset, validation_dataset = random_split(torch_dataset, [train_size, validation_size])

# # Create dataloaders for both the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=utils.collate_fn)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)


In [10]:
print(train_dataset[0])

(tensor([[[0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412],
         [0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412],
         [0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412],
         ...,
         [0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412],
         [0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412],
         [0.1412, 0.1412, 0.1412,  ..., 0.1412, 0.1412, 0.1412]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0

In [11]:
i=0
for imgs, targets in validation_dataloader:
    i+=1

print(i)

print(len(validation_dataset.samples))

j=0
for imgs, targets in train_dataloader:
    j+=1

print(j)
print(len(train_dataset.samples))

31
124
121
484


In [12]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/faster_rcnn_base')


In [13]:
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image

def eval_image(model, epoch):

    image = read_image("/home/zhiyi/fiftyone/coco-2017/train/data/000000027075.jpg")
    
    eval_transform = get_transform(train=False)
    
    model.eval()
    with torch.no_grad():
        x = eval_transform(image)
        # convert RGBA -> RGB and move to device
        x = x[:3, ...].to(device)
        predictions = model([x, ])
        pred = predictions[0]
    
    # print(pred) 
    for key, value in pred.items():
        pred[key] = pred[key][:3]
    
    # image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
    image = image[:3, ...]
    pred_labels = [f"cat: {score:.3f}" for label, score in zip(pred["labels"], pred["scores"])]
    pred_boxes = pred["boxes"].long()
    output_image = draw_bounding_boxes(image, pred_boxes, pred_labels, colors="red")
    
    # for label, box in zip(pred_labels, pred_boxes):
    #     print("label:", label, "box:", box)
    
    plt.figure(figsize=(12, 12))
    plt.imshow(output_image.permute(1, 2, 0))
    plt.savefig(f"eval_image/epoch{epoch}.jpg", bbox_inches='tight', pad_inches=0)  # Specify the path and filename
    plt.close() 

In [14]:
import custom_faster_rcnn as c_frcnn
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

if not os.path.isfile('fasterrcnn_resnet50_fpn_coco-258fb6c6.pth'):
    os.system('wget https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth')

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

custom = False

if custom:

    model = c_frcnn.custom_fasterrcnn_resnet50_fpn(num_classes=2)#weights=torchvision.models.detection.faster_rcnn.FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
    pretrained_dict = torch.load('fasterrcnn_resnet50_fpn_coco-258fb6c6.pth')  # Load the pretrained model weights
    model_dict = model.state_dict()
    
    # Filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and model_dict[k].shape == v.shape}
    model_dict.update(pretrained_dict) 
    model.load_state_dict(model_dict)
    overwrite_eps(model, 0.0)

    model.to(device)

else:

    # CREATE BASE_MODEL
   
    #model = fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.faster_rcnn.FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    
    # model.roi_heads.box_predictor = FastRCNNPredictor(in_channels=1024,num_classes=2)
    
    model.to(device)
    

# if torch.cuda.is_available():
#     # If a GPU is available, move the model to the GPU
#     model = model.cuda()

eval_image(model, -1)


In [15]:
import engine

num_epochs = 20

# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# figure out why class_logits is still rly long

if custom:

    for epoch in range(num_epochs):
        # train the model for one epoch
        train_metrics = c_frcnn.train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=500)
    
        writer.add_scalar('Loss/train', train_metrics.loss.avg, epoch)
        
        # update the learning rate
        lr_scheduler.step()
        
        # evaluate on the test dataset
        validation_metrics = c_frcnn.evaluate(model, validation_dataloader, device=device)
        writer.add_scalar('mAP/validation', validation_metrics.coco_eval['bbox'].stats[0], epoch)
        
        # save image of example
        eval_image(model, epoch)

else:
    
    for epoch in range(num_epochs):
        # train the model for one epoch
        train_metrics=engine.train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=250)
        writer.add_scalar('Loss/train', train_metrics.loss.global_avg, epoch)
        
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        validation_metrics=engine.evaluate(model, validation_dataloader, device=device)
        writer.add_scalar('mAP/validation', validation_metrics.coco_eval['bbox'].stats[0], epoch)
    
        eval_image(model, epoch)

        # Additional information
        checkpoint_file = f"./checkpoints_stop/model_{epoch}.pt"
        
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': train_metrics.loss.global_avg
                    }, checkpoint_file)

Epoch: [0]  [  0/121]  eta: 0:01:59  lr: 0.000047  loss: 0.3614 (0.3614)  loss_classifier: 0.3143 (0.3143)  loss_box_reg: 0.0296 (0.0296)  loss_objectness: 0.0004 (0.0004)  loss_rpn_box_reg: 0.0170 (0.0170)  time: 0.9907  data: 0.0255  max mem: 4885
Epoch: [0]  [120/121]  eta: 0:00:00  lr: 0.005000  loss: 0.0937 (0.1469)  loss_classifier: 0.0278 (0.0657)  loss_box_reg: 0.0525 (0.0596)  loss_objectness: 0.0025 (0.0039)  loss_rpn_box_reg: 0.0161 (0.0176)  time: 0.8543  data: 0.0216  max mem: 6164
Epoch: [0] Total time: 0:01:42 (0.8456 s / it)
creating index...
index created!
Test:  [ 0/31]  eta: 0:00:13  model_time: 0.4114 (0.4114)  evaluator_time: 0.0012 (0.0012)  time: 0.4436  data: 0.0232  max mem: 6164
Test:  [30/31]  eta: 0:00:00  model_time: 0.3231 (0.3591)  evaluator_time: 0.0012 (0.0012)  time: 0.3845  data: 0.0205  max mem: 6164
Test: Total time: 0:00:12 (0.3878 s / it)
Averaged stats: model_time: 0.3231 (0.3591)  evaluator_time: 0.0012 (0.0012)
Accumulating evaluation results..

In [16]:
writer.flush()
writer.close()

# print(train_metrics.loss.avg)
# print(validation_metrics.coco_eval['bbox'].stats[0])

In [17]:
# END

# print(train_metrics.loss.global_avg)
shutdown=False

if shutdown:

    import os

    os.system('shutdown -h now')

In [18]:
# END

In [19]:
%%script echo skipping  
# plot bbox from model
#file = "/home/zhiyi/fiftyone/open-images-v7/validation/data/0f426802e98a8964.jpg"
#file = "/home/zhiyi/fiftyone/open-images-v7/train/data/0ca0f9c12dde03e0.jpg"
file = '/home/zhiyi/fiftyone/coco-2017/train/data/000000000064.jpg'
image = read_image(file)
model.eval()
with torch.no_grad():
    eval_transform = get_transform(train=False)
    x = eval_transform(image)
    # convert RGBA -> RGB and move to device
    x = x[:3, ...].to(device)
    predictions = model([x, ])
    pred = predictions[0]

# print(pred)

for key, value in pred.items():
    pred[key] = pred[key][:1]

image = (255.0 * (image - image.min()) / (image.max() - image.min())).to(torch.uint8)
image = image[:3, ...]
pred_labels = [f"cat: {score:.3f}" for label, score in zip(pred["labels"], pred["scores"])]
pred_boxes = pred["boxes"].long()
output_image = draw_bounding_boxes(image, pred_boxes, pred_labels, colors="red")

for label, box in zip(pred_labels, pred_boxes):
    print("label:", label, "box:", box)

plt.figure(figsize=(12, 12))
plt.imshow(output_image.permute(1, 2, 0))

skipping


In [20]:
%%script echo skipping 
#TARGET BBOX

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def plot_image_with_boxes(dataset, image_file_name):
    # Find the sample in the dataset
    sample = next((s for s in dataset.samples if s.filepath == image_file_name), None)
    if sample is None:
        print(f"Image {image_file_name} not found in the dataset.")
        return

    # Load the image
    img = Image.open(sample.filepath)
    img_width, img_height = img.size
    
    # Set up the plot
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)

    # Plot each bounding box
    detections = sample[dataset.gt_field].detections
    for det in detections:
        # Scale bounding box coordinates to image size
        x, y, width, height = det.bounding_box
        x, width = x * img_width, width * img_width
        y, height = y * img_height, height * img_height

        rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, det.label, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()

plot_image_with_boxes(train_dataset, "/home/zhiyi/fiftyone/coco-2017/train/data/000000000064.jpg")

skipping


In [21]:
%%script echo skipping 
import matplotlib.patches as patches
import matplotlib.pyplot as plt

test_dataset = FiftyOneTorchDataset(validation_dataset_view, transforms=T.ToDtype(torch.float, scale=True), classes=classes_used)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)


i = 0
for images, targets in test_dataloader:
    tensor_image = images[0]
    # print(tensor_image.size())
    _, img_height, img_width = tensor_image.size()
    target = targets[0]
    tensor_image = tensor_image.permute(1, 2, 0)
    numpy_image = tensor_image.numpy()
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(numpy_image)
    
    image_id = target['image_id']
    image_path = test_dataset.img_paths[image_id]
    # print(validation_dataset_view[image_path])
    
    # img = Image.open(image_path)
    
    for box in target['boxes']:
        # Scale bounding box coordinates to image size
        x1,y1,x2,y2 = box
        width = x2-x1
        height=y2-y1
        # x1,y1,width,height=box

        # print((x1,y1,width,height))
        
        
        # x, y, width, height = box
        # x, width = x * img_width, width * img_width
        # y, height = y * img_height, height * img_height

        rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1, 'cat', color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()
    i+=1
    if i>2:
        break

skipping


In [22]:
%%script echo skipping 
img = read_image("/home/zhiyi/fiftyone/open-images-v7/validation/data/0f426802e98a8964.jpg")
eval_transform = get_transform(train=False)
print(img.max())
print(img.min())
print(img.dtype)
img = eval_transform(img)
print(img.max())
print(img.min())
# img_width, img_height = img.size()[1:]
# print(img.size())
# print(img_width)
# print(type(img_width))

# num_epochs = 3

# # train on the GPU or on the CPU, if a GPU is not available
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(
#     params,
#     lr=0.005,
#     momentum=0.9,
#     weight_decay=0.0005
# )

# for epoch in range(num_epochs):
#   train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=10)

skipping


In [23]:
%%script echo skipping 
image_paths = dataset.values("filepath")
sample = dataset[image_paths[0]]
print(sample)
detections = sample["ground_truth"].detections
metadata = sample.metadata
for det in detections:
    # category_id = self.labels_map_rev[det.label]
    category_id = 0
    print(det.label)
    print(det.IsOccluded)
    print(list(det.bounding_box))
    print(type(list(det.bounding_box)))
    print(torch.as_tensor(list(det.bounding_box))[1])

img = Image.open(image_paths[0]).convert("RGB")
img = torchvision.transforms.ToTensor()(img)
print(type(img))
    # coco_obj = fouc.COCOObject.from_label(
    #     det, metadata, category_id=category_id,
    # )
    # x, y, w, h = coco_obj.bbox
    # print(coco_obj.category_id)
    # print(coco_obj.area)
    # print(coco_obj.isoccluded)


skipping


In [24]:
%%script echo skipping 
img_path = '/home/zhiyi/fiftyone/coco-2017/train/data/000000000064.jpg'
img = read_image(img_path, mode=ImageReadMode.RGB)
img_height, img_width = img.size()[1:]   
print(img_height, img_width)

skipping


In [25]:
%%script echo skipping 
import torch
from PIL import Image
from torchvision import tv_tensors
# import fiftyone.utils.coco as fouc
from torchvision.io import read_image, ImageReadMode

class FiftyOneOpenImagesTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.

    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for
            training or testing
        transforms (None): a list of PyTorch transforms to apply to images
            and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset
            that contains the desired labels to load
        classes (None): a list of class strings that are used to define the
            mapping between class names and indices. If None, it will use
            all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.filtered_indices = []
        for sample in fiftyone_dataset:
            detections = sample[gt_field].detections
            if detections and any(self.is_valid_box(det.bounding_box) for det in detections):
                self.filtered_indices.append(sample.filepath)

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    @staticmethod
    def is_valid_box(box):
        # Check if the bounding box is valid (non-zero area)
        x1, y1, x2, y2 = box
        x_min, x_max = min(x1,x2), max(x1,x2)
        y_min, y_max = min(y1,y2), max(y1,y2)
        return (x_max > x_min) and (y_max > y_min)
    
    def __getitem__(self, idx):
        # img_path = self.img_paths[idx]
        img_path = self.filtered_indices[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        #img = Image.open(img_path).convert("RGB")
        img = read_image(img_path, mode=ImageReadMode.RGB)
        img_height, img_width = img.size()[1:]        

        boxes = []
        labels = []
        areas = []
        isoccluded = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            box = list(det.bounding_box)
            #corrected_box = box.clone()

            # fiftyone dataset stores bbox as [0,1] values, and as XYWH format?
            
            box[0], box[2] = min(box[0], box[2]), max(box[0], box[2])
            box[1], box[3] = min(box[1], box[3]), max(box[1], box[3])
            absolute_box = [box[0]*img_width, box[1]*img_height, box[2]*img_width, box[3]*img_height]

            area = (absolute_box[2]-absolute_box[0])*(absolute_box[3]-absolute_box[1])
            # area = (box[2]-box[0])*(box[3]-box[1])
            
            if box[2] > box[0] and box[3] > box[1]: # make sure the box is valid
                boxes.append(absolute_box)
                labels.append(category_id)
                # isoccluded.append(det.IsOccluded)
                areas.append(area)
                iscrowd.append(0)

        target = {}
        # target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="XYXY", canvas_size=(img_height, img_width), dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        # target["attributes"] = torch.as_tensor(isoccluded, dtype=torch.int64)
        target["image_id"] = idx #torch.as_tensor([idx])
        target["area"] = torch.as_tensor(areas, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        img = tv_tensors.Image(img)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.filtered_indices)
        # return len(self.img_paths)

    def get_classes(self):
        return self.classes


        # image_paths = dataset.values("filepath")
        # sample = dataset[image_paths[1]]
        # print(sample)
        # detections = sample["ground_truth"].detections
        # metadata = sample.metadata
        # for det in detections:
        #     # category_id = self.labels_map_rev[det.label]
        #     category_id = 0
        #     print(det.label)
        #     print(det.IsOccluded)
        #     print(det.bounding_box)
        #     print(type(det.bounding_box))
            # coco_obj = fouc.COCOObject.from_label(
            #     det, metadata, category_id=category_id,
            # )
            #areas.append(det.area)

skipping
